In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/360_metrics

Mounted at /content/drive
/content/drive/MyDrive/Github/360_metrics


In [ ]:
%cd 360SR-Challenge/scripts

/content/drive/MyDrive/Github/360_metrics/360SR-Challenge/scripts


In [ ]:
!pip install ffmpeg-python
!pip install glymur

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 55.0 MB/s eta 0:00:00


In [ ]:
import glymur
import ffmpeg
import cv2
import numpy as np
from PIL import Image
import os
import pickle
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [ ]:
#img_name = "P29"

def compress_images(img_list):
  evaluation_dict = {}
  for img_name_full in tqdm(img_list):
    img_name = img_name_full.split(".")[0]
    original_extension = img_name_full.split(".")[1]

    evaluation_dict[img_name] = {}
    evaluation_dict[img_name]["HEVC"] = {}
    evaluation_dict[img_name]["JPEG"] = {}
    evaluation_dict[img_name]["JPEG2000"] = {}

    #HEVC, JPEG, JPEG2000 
    img = ffmpeg.input(f'eval/{img_name}.{original_extension}')
    compressed_img = ffmpeg.output(img, f'./compressed_{img_name}.hevc', vcodec='libx265', pix_fmt='yuv420p')
    ffmpeg.run(compressed_img)
    decompressed_img = ffmpeg.input(f'./compressed_{img_name}.hevc')
    decompressed_img = ffmpeg.output(decompressed_img, f'compressed/HEVC/hevc_compressed_{img_name}.jpg', pix_fmt='rgb24')
    ffmpeg.run(decompressed_img)
    
    # Open the image
    img_hevc = Image.open(f'compressed/HEVC/hevc_compressed_{img_name}.jpg')

    # Load the image
    img_original = Image.open(f'eval/{img_name}.{original_extension}')

    # Compress using JPEG and save to file
    img_original.save(f'compressed/JPEG/JPEG_compressed_{img_name}.jpg', quality=30)

    # Compress using JPEG2000 and save to file
    glymur.Jp2k(f'compressed/JPEG2000/JPEG2000_compressed_{img_name}.jp2',data=np.array(img_original), cratios=[70])

    # Load the compressed image
    # Get the total size of the image in bytes calculate the bpp
    total_size = os.path.getsize(f'compressed/HEVC/hevc_compressed_{img_name}.jpg')
    bpp = (total_size * 8) / (len(img_hevc.getbands())*img_hevc.width * img_hevc.height)
    psnr_value = psnr(np.array(img_original), np.array(img_hevc))
    ssim_value = ssim(np.array(img_original), np.array(img_hevc), multichannel=True)
    evaluation_dict[img_name]["HEVC"]["ssim"] = ssim_value
    evaluation_dict[img_name]["HEVC"]["psnr"] = psnr_value
    evaluation_dict[img_name]["HEVC"]["bpp"] = bpp

    compressed_image = Image.open(f'compressed/JPEG/JPEG_compressed_{img_name}.jpg')
    compressed_image_array = np.array(compressed_image)
    total_size = os.path.getsize(f'compressed/JPEG/JPEG_compressed_{img_name}.jpg')
    bpp = (total_size * 8) / (len(compressed_image.getbands())*compressed_image.width * compressed_image.height)
    psnr_value = psnr(np.array(img_original), compressed_image_array)
    ssim_value = ssim(np.array(img_original), compressed_image_array, multichannel=True)
    evaluation_dict[img_name]["JPEG"]["ssim"] = ssim_value
    evaluation_dict[img_name]["JPEG"]["psnr"] = psnr_value
    evaluation_dict[img_name]["JPEG"]["bpp"] = bpp

    compressed_image = Image.open(f'compressed/JPEG2000/JPEG2000_compressed_{img_name}.jp2')
    compressed_image_array = np.array(compressed_image)
    total_size = os.path.getsize(f'compressed/JPEG2000/JPEG2000_compressed_{img_name}.jp2')
    bpp = (total_size * 8) / (len(compressed_image.getbands())*compressed_image.width * compressed_image.height)
    psnr_value = psnr(np.array(img_original), compressed_image_array)
    ssim_value = ssim(np.array(img_original), compressed_image_array, multichannel=True)
    evaluation_dict[img_name]["JPEG2000"]["ssim"] = ssim_value
    evaluation_dict[img_name]["JPEG2000"]["psnr"] = psnr_value
    evaluation_dict[img_name]["JPEG2000"]["bpp"] = bpp

    os.remove(f'./compressed_{img_name}.hevc')
    #os.remove(f'compressed/hevc_compressed_{img_name}.jpg')
    #os.remove(f'compressed/JPEG_compressed_{img_name}.jpg')
    #os.remove(f'compressed/JPEG2000_compressed_{img_name}.jp2')

    with open('compression_info_eval.pickle', 'wb') as handle:
        pickle.dump(evaluation_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


  return evaluation_dict

In [ ]:
original_img_list = os.listdir("./eval/")
eval_dict = compress_images(original_img_list)

100%|██████████| 25/25 [25:10<00:00, 60.44s/it]


In [ ]:
with open('compression_info_eval.pickle', 'rb') as handle:
    eval_dict = pickle.load(handle)

eval_dict

In [ ]:
from cal_ws_metrics import cal_ws_metrics 
src_gt = './eval'
src_sr = './compressed/JPEG'
src_sal = "./saliency"
metrics = ['wspsnr', 'wsssim',"salpsnr","psnr","ssim","bpp"]
cal_ws_metrics(src_gt, src_sr, src_sal, metrics)

[0/25][idx]:	wspsnr:31.45	wsssim:0.8743	salpsnr:30.5335	psnr:32.1600	ssim:0.4677	bpp:0.1410
[1/25][idx]:	wspsnr:26.45	wsssim:0.7090	salpsnr:25.8041	psnr:26.9826	ssim:0.4464	bpp:0.2144
[2/25][idx]:	wspsnr:33.19	wsssim:0.8887	salpsnr:32.2747	psnr:34.0107	ssim:0.4722	bpp:0.1281
[3/25][idx]:	wspsnr:27.47	wsssim:0.7926	salpsnr:27.0046	psnr:28.2710	ssim:0.5380	bpp:0.2207
[4/25][idx]:	wspsnr:30.45	wsssim:0.8480	salpsnr:30.0772	psnr:31.2791	ssim:0.5668	bpp:0.1741
[5/25][idx]:	wspsnr:36.08	wsssim:0.9309	salpsnr:35.7450	psnr:36.9360	ssim:0.4057	bpp:0.0983
[6/25][idx]:	wspsnr:34.81	wsssim:0.9127	salpsnr:34.3975	psnr:35.4495	ssim:0.5812	bpp:0.1378
[7/25][idx]:	wspsnr:35.91	wsssim:0.9191	salpsnr:35.1762	psnr:36.3872	ssim:0.5631	bpp:0.1094
[8/25][idx]:	wspsnr:29.81	wsssim:0.8511	salpsnr:29.2053	psnr:30.7276	ssim:0.6497	bpp:0.2240
[9/25][idx]:	wspsnr:38.05	wsssim:0.9383	salpsnr:37.1715	psnr:38.5974	ssim:0.3515	bpp:0.0768
[10/25][idx]:	wspsnr:33.73	wsssim:0.9148	salpsnr:32.9823	psnr:34.5771	ssim:0.616

{'wspsnr': {'P1': 31.44981107206892,
  'P16': 26.448114671095794,
  'P19': 33.186597271774275,
  'P20': 27.474630335351186,
  'P26': 30.447088527708686,
  'P48': 36.07567289195339,
  'P50': 34.806695907802535,
  'P60': 35.91452210628739,
  'P65': 29.814394200854345,
  'P69': 38.05179242993961,
  'P71': 33.730762540909744,
  'P72': 29.765795218256407,
  'P73': 32.957862258827,
  'P74': 33.08390095379971,
  'P78': 35.98791189359344,
  'P79': 29.50540323284851,
  'P8': 34.04621145175478,
  'P85': 31.44417398530343,
  'P91': 31.771625981316937,
  'P93': 31.833288875187943,
  'P94': 36.749761832487785,
  'P95': 32.59328409379123,
  'P96': 35.54774521506878,
  'P97': 34.63549783549721,
  'P98': 36.022848089398096},
 'wspsnr_avg': 32.933815714915085,
 'wsssim': {'P1': 0.8743312363918618,
  'P16': 0.7089643124181676,
  'P19': 0.8886962567089389,
  'P20': 0.7925826742011486,
  'P26': 0.8480228256410594,
  'P48': 0.9309046752158586,
  'P50': 0.9127395322935857,
  'P60': 0.919085749939261,
  'P65

In [ ]:
from cal_ws_metrics import cal_ws_metrics
src_gt = './eval'
src_sr = './compressed/JPEG2000'
src_sal = "./saliency"
metrics = ['wspsnr', 'wsssim',"salpsnr","psnr","ssim","bpp"]
cal_ws_metrics(src_gt, src_sr, src_sal, metrics)

[0/25][idx]:	wspsnr:34.15	wsssim:0.9113	salpsnr:33.6903	psnr:34.5974	ssim:0.4758	bpp:0.1142
[1/25][idx]:	wspsnr:27.01	wsssim:0.6858	salpsnr:26.7579	psnr:27.3085	ssim:0.3476	bpp:0.1143
[2/25][idx]:	wspsnr:35.34	wsssim:0.9085	salpsnr:34.9819	psnr:35.8610	ssim:0.4444	bpp:0.1143
[3/25][idx]:	wspsnr:27.69	wsssim:0.7378	salpsnr:27.5603	psnr:28.2132	ssim:0.4039	bpp:0.1143
[4/25][idx]:	wspsnr:31.50	wsssim:0.8424	salpsnr:31.3745	psnr:32.0308	ssim:0.4886	bpp:0.1143
[5/25][idx]:	wspsnr:39.81	wsssim:0.9552	salpsnr:39.5515	psnr:40.4947	ssim:0.4646	bpp:0.1143
[6/25][idx]:	wspsnr:36.33	wsssim:0.9254	salpsnr:36.1495	psnr:36.8318	ssim:0.5651	bpp:0.1142
[7/25][idx]:	wspsnr:38.87	wsssim:0.9422	salpsnr:38.5389	psnr:39.3191	ssim:0.6211	bpp:0.1142
[8/25][idx]:	wspsnr:28.86	wsssim:0.8119	salpsnr:28.6230	psnr:29.5591	ssim:0.5363	bpp:0.1143
[9/25][idx]:	wspsnr:42.29	wsssim:0.9650	salpsnr:41.8311	psnr:42.7268	ssim:0.5652	bpp:0.1143
[10/25][idx]:	wspsnr:35.27	wsssim:0.9264	salpsnr:34.7495	psnr:35.9729	ssim:0.624

{'wspsnr': {'P1': 34.14532855430484,
  'P16': 27.006770655736876,
  'P19': 35.337455873227725,
  'P20': 27.685362352901773,
  'P26': 31.502839806748565,
  'P48': 39.81297137054444,
  'P50': 36.3297271477754,
  'P60': 38.86734870097865,
  'P65': 28.864605048461776,
  'P69': 42.286596739277385,
  'P71': 35.271680950654996,
  'P72': 30.906686972072105,
  'P73': 36.19920450074913,
  'P74': 33.10428470299436,
  'P78': 38.80833301509821,
  'P79': 31.35366293608513,
  'P8': 36.31285684314777,
  'P85': 33.08188702998954,
  'P91': 31.778679202432354,
  'P93': 30.67913847645289,
  'P94': 41.04446171269724,
  'P95': 33.026190378920035,
  'P96': 39.79678427277599,
  'P97': 38.36794724597583,
  'P98': 39.64918789465419},
 'wspsnr_avg': 34.84879969538628,
 'wsssim': {'P1': 0.9113025472130474,
  'P16': 0.685820020995726,
  'P19': 0.9084696261090572,
  'P20': 0.7377626703517598,
  'P26': 0.8424117476440008,
  'P48': 0.9552206961994552,
  'P50': 0.9253799427526298,
  'P60': 0.9421907707387752,
  'P65':

In [ ]:
src_gt = './eval'
src_sr = './compressed/HEVC'
src_sal = "./saliency"
metrics = ['wspsnr', 'wsssim',"salpsnr","psnr","ssim","bpp"]
cal_ws_metrics(src_gt, src_sr, src_sal, metrics)

[0/25][idx]:	wspsnr:31.97	wsssim:0.8740	salpsnr:31.2267	psnr:32.6656	ssim:0.4026	bpp:0.1693
[1/25][idx]:	wspsnr:27.03	wsssim:0.6971	salpsnr:26.6131	psnr:27.5844	ssim:0.3907	bpp:0.2562
[2/25][idx]:	wspsnr:32.46	wsssim:0.8677	salpsnr:31.8263	psnr:33.3956	ssim:0.3577	bpp:0.1355
[3/25][idx]:	wspsnr:27.95	wsssim:0.7667	salpsnr:27.6643	psnr:28.6509	ssim:0.4722	bpp:0.2443
[4/25][idx]:	wspsnr:29.34	wsssim:0.7970	salpsnr:29.0083	psnr:30.0935	ssim:0.4319	bpp:0.1450
[5/25][idx]:	wspsnr:34.94	wsssim:0.9112	salpsnr:34.5961	psnr:36.0700	ssim:0.2798	bpp:0.0914
[6/25][idx]:	wspsnr:33.54	wsssim:0.8763	salpsnr:33.2595	psnr:34.2435	ssim:0.4760	bpp:0.1384
[7/25][idx]:	wspsnr:35.25	wsssim:0.9045	salpsnr:34.5173	psnr:35.7585	ssim:0.4250	bpp:0.1194
[8/25][idx]:	wspsnr:27.84	wsssim:0.7746	salpsnr:27.4745	psnr:28.6630	ssim:0.5033	bpp:0.1658
[9/25][idx]:	wspsnr:37.67	wsssim:0.9320	salpsnr:36.6792	psnr:38.5493	ssim:0.2614	bpp:0.0805
[10/25][idx]:	wspsnr:32.25	wsssim:0.8786	salpsnr:31.5826	psnr:33.1361	ssim:0.508

{'wspsnr': {'P1': 31.96953906014209,
  'P16': 27.034503558749485,
  'P19': 32.46399826319896,
  'P20': 27.947001260369394,
  'P26': 29.338337743812698,
  'P48': 34.94241773515006,
  'P50': 33.5371877622194,
  'P60': 35.24636716020518,
  'P65': 27.838569230038313,
  'P69': 37.67120258587384,
  'P71': 32.24757266240243,
  'P72': 29.284085890111378,
  'P73': 31.968359905735717,
  'P74': 31.10840223879862,
  'P78': 35.60421937982416,
  'P79': 29.506875954567548,
  'P8': 33.68907527416356,
  'P85': 30.67791316429734,
  'P91': 30.157999383009184,
  'P93': 29.900042774810096,
  'P94': 36.323776954707085,
  'P95': 31.15976266800308,
  'P96': 34.45436343266214,
  'P97': 33.45138962343745,
  'P98': 34.81866649573791},
 'wspsnr_avg': 32.09366520648109,
 'wsssim': {'P1': 0.8739892987926067,
  'P16': 0.6970672998261728,
  'P19': 0.8677153780191803,
  'P20': 0.7667030240956079,
  'P26': 0.7969936832930347,
  'P48': 0.9111655902220105,
  'P50': 0.8763044936888482,
  'P60': 0.9045250713664291,
  'P65'